In [25]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import csv
import pandas as pd
import utils
import re
import decimal
import optparse

print(sklearn.__version__)

0.19.0


In [30]:
def parseNumbers( s):
        """
        Parses a string that represents a number into a decimal data type so that
        we can match the quantity field in the db with the quantity that appears
        in the display name. Rounds the result to 2 places.
        """
        ss = utils.unclump(s)

        m3 = re.match('^\d+$', ss)
        if m3 is not None:
            return decimal.Decimal(round(float(ss), 2))

        m1 = re.match(r'(\d+)\s+(\d)/(\d)', ss)
        if m1 is not None:
            num = int(m1.group(1)) + (float(m1.group(2)) / float(m1.group(3)))
            return decimal.Decimal(str(round(num,2)))

        m2 = re.match(r'^(\d)/(\d)$', ss)
        if m2 is not None:
            num = float(m2.group(1)) / float(m2.group(2))
            return decimal.Decimal(str(round(num,2)))

        return None
    
    

def matchUp( token, ingredientRow):
        """
        Returns our best guess of the match between the tags and the
        words from the display text.
        This problem is difficult for the following reasons:
            * not all the words in the display name have associated tags
            * the quantity field is stored as a number, but it appears
              as a string in the display name
            * the comment is often a compilation of different comments in
              the display name
        """
        ret = []

        # strip parens from the token, since they often appear in the
        # display_name, but are removed from the comment.
        token = utils.normalizeToken(token)
        decimalToken = parseNumbers(token)

        for key, val in ingredientRow.iteritems():
            if isinstance(val, str):

                for n, vt in enumerate(utils.tokenize(val)):
                    if utils.normalizeToken(vt) == token:
                        ret.append(key.upper())

            elif decimalToken is not None:
                try:
                    if val == decimalToken:
                        ret.append(key.upper())
                except:
                    pass

        return ret
    
    
    
def addPrefixes( data):
        """
        We use BIO tagging/chunking to differentiate between tags
        at the start of a tag sequence and those in the middle. This
        is a common technique in entity recognition.
        Reference: http://www.kdd.cis.ksu.edu/Courses/Spring-2013/CIS798/Handouts/04-ramshaw95text.pdf
        """
        prevTags = None
        newData = []

        for n, (token, tags) in enumerate(data):

            newTags = []

            for t in tags:
                p = "B" if ((prevTags is None) or (t not in prevTags)) else "I"
                newTags.append("%s-%s" % (p, t))

            newData.append((token, newTags))
            prevTags = tags

        return newData


def bestTag( tags):

        if len(tags) == 1:
            return tags[0]

        # if there are multiple tags, pick the first which isn't COMMENT
        else:
            for t in tags:
                if (t != "B-COMMENT") and (t != "I-COMMENT"):
                    return t

        # we have no idea what to guess
        return "OTHER"
    


# Building the NER system

### Obtaining the dataset
And parse it to a format which we the trainer can read

In [40]:
# -*- coding: utf-8 -*
csv_path = 'nyt-ingredients-snapshot-2015.csv'

cnt = 0

df = pd.read_csv(csv_path)
df = df.fillna("")

df_slice = df.iloc[:]

with open(r"training_file",'w',encoding='utf8') as file:
    for index, row in df_slice.iterrows():
        try:
            # extract the display name
            display_input = utils.cleanUnicodeFractions(row["input"])
            tokens = list(utils.tokenize(display_input))
            del(row["input"])

            rowData = addPrefixes([(t, matchUp(t, row)) for t in tokens])

            for i, (token, tags) in enumerate(rowData):
                features = utils.getFeatures(token, i+1, tokens)
                file.write(str( utils.joinLine([token] + features + [bestTag(tags)])) + '\n')

        # ToDo: deal with this
        except UnicodeDecodeError:
            pass

        file.write('\n')

C:\Users\peter\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [ ]:
raw_lines[432]